In [4]:
"""
This code is for comparing a dataset of located weather system centers with the 
dataset of potential vorticity cutoffs published in Portmann et al. 2021 
(https://doi.org/10.5194/wcd-2-507-2021). 
This code computes how many weather systems are located within the areas of 
stratospheric potential vorticity (hits), and how many are located outside (misses). 
"""
import numpy as np
import xarray as xr
from datetime import datetime
import matplotlib.dates as dt

In [5]:
def fourxdaily_timestep(y,m,d,h,y0):
    
    '''This function takes a date in y,m,d,h format and returns the value of the
    hours since jan 1, y0 (the time convention of reanalysis). Inputs are integers. 
    Output is a float.''' 
    
    d0 = datetime(y0,1,1,0) # the datetime object representing 12am, jan 1 of the baseline year of the reanalysis
    d = datetime(y,m,d,h) # the datetime object for the day and time being tested
    
    t = int(4 * (dt.date2num(d) - dt.date2num(d0)) )# number of timesteps since 12am, jan 1 of the baseline year
    # dt.date2num gives the number of days since the python datetime time origin. 
    
    return t

In [7]:
save_results = 1

years = (1979, 2017)
months = (1,12)

save_path = '/home/561/nxg561/00_Tracking_Scheme_Comparison/Output_Data/MUvsPV/'

In [8]:
# Open the array of Portmann fields and extract the numpy array
portmann_dataset_filename = '/g/data/w40/nxg561/Portmann_Cutoffs/300_320/pv_cutoffs_1979-2017.nc'
xr_obj_pm = xr.open_dataset(portmann_dataset_filename, engine = 'netcdf4')

pm_array = xr_obj_pm['track_id'].data

0.3.0


In [9]:
# This part computes the relevant offset value for the latitudes of the cyclone database to 
# correspond to indices of the Portmann's array. 
pm_lats = xr_obj_pm['latitude'].data
lat_offset = pm_lats[0]
lat_offset

-90

In [10]:
# Open the dataset of lows identified by the melbourne uni scheme
dataset0_filename = '/home/561/nxg561/00_Tracking_Scheme_Comparison/Input_Data/Acacia_Lows/closed_lows_mu_era5_2017.txt'
dataset0 = np.loadtxt(dataset0_filename, delimiter = ',')

#loop through the rows of dataset0 and convert the timestep to the rows of the Portmann's array
rows = np.shape(dataset0)[0]

lat_inds = np.zeros((rows))
lon_inds = np.zeros((rows))

for row in range(rows):
    
    dataset0[row,5] = fourxdaily_timestep(int(dataset0[row,1]),int(dataset0[row,2]),int(dataset0[row,3]),int(dataset0[row,4]),1979)

    # add an offset to the lat and lon values so they become indices of the Portmann's array
    lat_inds[row] =  int(dataset0[row,6]) - lat_offset
    lon_inds[row] =  int(dataset0[row,7]) + 180

time_inds = dataset0[:,5]

In [11]:
# Extract all the data elements of the Portmann's array at each of the time, lat and lon 
# points of the lows reccorded in dataset0. The value will be greater than 0 at a point
# where Portmann's dataset reccords a pv cutoff. 
matcharray = pm_array[time_inds.astype(int), lat_inds.astype(int), lon_inds.astype(int)]
#print(np.shape(pm_array))
#print(np.shape(matcharray))

In [12]:
hits_inds = np.where(matcharray > 0)
extra_inds = np.where(matcharray == 0)

hits = np.shape(hits_inds)[1]
extras = np.shape(extra_inds)[1]

print(hits_inds[0])
print(extra_inds[0])

[    1     2     3 ... 43784 43785 43786]
[    0     7     8 ... 43781 43782 43783]


In [13]:
# Compute the percentage of closed lows from dataset0 that match pv cutoffs
# in Portmann's dataset.
percent_matches = hits / rows
print('MU Percent matches:')
print(percent_matches)

MU Percent matches:
0.5789846301413661


In [14]:
print('Matches per year:')
print(hits/ (years[1] - years[0] + 1))
print('Extras per year:')
print(extras/ (years[1] - years[0] + 1))

Matches per year:
650.0512820512821
Extras per year:
472.6923076923077


In [15]:
if save_results:

    # save the output
    np.savetxt(save_path + 'mu_matches.txt', dataset0[hits_inds[0],:], delimiter = ',')
    np.savetxt(save_path + 'mu_only.txt', dataset0[extra_inds[0],:], delimiter = ',')